# **STAR SCHEMA TEMPLATE**

## **_Schema DDL_**

In [35]:
IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'stg' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA stg AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;

GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.008

**Simplification :**    

<span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; font-weight: 700; white-space: pre;">/* </span> <span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; font-weight: bold; white-space: pre;">Checks if 'dim' schema existence and creates if not exists.</span><span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; font-weight: bold; white-space: pre;">*/</span>

<span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; font-weight: bold; white-space: pre;">/* Check for 'stg' schema existence and create if not exists */</span>

<span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; font-weight: bold; white-space: pre;">/* Check for 'f' schema existence and create if not exists */</span>

## **_Customer Dim Script_**

In [36]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Customers')
BEGIN

--Create the Customer table

	CREATE TABLE dim.Customers(
	pkCustomer int IDENTITY(1000,1) NOT NULL,
	CustomerID nvarchar(5) NOT NULL,
	Customer nvarchar(40) NOT NULL,
	City nvarchar(15) NULL,
	Country nvarchar(15) NULL,
	LoadDate DATE NOT NULL,
	SourceCountry nvarchar(25) NULL
	)
	;
	
        -- Adds primary and unique constraints to the table.

	ALTER TABLE dim.Customers
	ADD CONSTRAINT PK_Customers_LUP PRIMARY KEY(pkCustomer);

	ALTER TABLE dim.Customers
        ADD CONSTRAINT UC_Customers_ID UNIQUE (CustomerID);

END

Commands completed successfully.

Total execution time: 00:00:00.022

## _**Calendar Dim Script**_

In [37]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')
BEGIN

-- Create the Calendar table

CREATE TABLE dim.Calendar
(
    pkCalendar INT NOT NULL,
    DateValue DATE NOT NULL,
    Year INT NOT NULL,
    Quarter INT NOT NULL,
	Qtr VARCHAR(3) NOT NULL,
    Month INT NOT NULL,
    MonthName VARCHAR(10) NOT NULL,
	MonthShort VARCHAR(3) NOT NULL,
    Week INT NOT NULL,
    Day INT NOT NULL,
	DayName VARCHAR(10) NOT NULL,
	DayShort VARCHAR(3) NOT NULL,
    IsWeekday BIT NOT NULL,
	Weekday VARCHAR(3) NOT NULL
);

--Adds primary and unique constraints to the table.
	
	ALTER TABLE dim.Calendar
	ADD CONSTRAINT PK_Calendar_Julian PRIMARY KEY(pkCalendar);

	ALTER TABLE dim.Calendar
         ADD CONSTRAINT UC_Calendar UNIQUE (DateValue);
END

Commands completed successfully.

Total execution time: 00:00:00.014

## _**Shipper Dim Script**_

In [38]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Shipper')
BEGIN

-- Create the Shipper table

CREATE TABLE dim.Shipper
(
    pkShipId int not null,
    Shipper nvarchar(40) not null,
    Phone nvarchar(24) null
);

-- Adds primary constraints to the table.
	
	ALTER TABLE dim.Shipper
	ADD CONSTRAINT PK_Ship PRIMARY KEY(pkShipId);
END

GO

Commands completed successfully.

Total execution time: 00:00:00.006

##  _**Products Dim Script**_

In [39]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Products')
BEGIN

--Create the Product table

CREATE TABLE dim.Products(
	pkProdId int NOT NULL,
	ProductId int NOT NULL,
	Product nvarchar(50) NOT NULL,
	ProductCategory nvarchar(25) NOT NULL,
	CategoryDesc ntext NULL,
	UnitPrice money NULL,
	UnitsInStock int NULL,
	UnitsOnOrder int NULL,
	ReorderLevel int NULL,
	Discontinued bit NOT NULL,
	ReorderFlag nvarchar(3) NOT NULL
);

 -- Adds primary and unique constraints to the table.
 
	ALTER TABLE dim.Products
	ADD CONSTRAINT PK_Prods PRIMARY KEY(pkProdId);

	ALTER TABLE dim.Products
    ADD CONSTRAINT UC_Prods UNIQUE (ProductId);
END
;

Commands completed successfully.

Total execution time: 00:00:00.009

## _**Employees Dim Script**_

In [40]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Employees')
BEGIN
-- Create the Employees table
CREATE TABLE dim.Employees(
	EmployeeID int NOT NULL,
	Employee nvarchar(50) NOT NULL,
	Title nvarchar(30) NULL,
	BirthDate datetime NULL,
	HireDate datetime NULL,
	City nvarchar(15) NULL,
	Country nvarchar(15) NULL,
	ReportsTo int NULL
);
-- Adds primary constraints to the table.

	ALTER TABLE dim.Employees
	ADD CONSTRAINT PK_Emp PRIMARY KEY(EmployeeID);

END

GO

Commands completed successfully.

Total execution time: 00:00:00.007

## **_Schema fact tables_**

### **_fact table 1st -_**  **_f.OrderPerf_**

In [41]:
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'OrderPerf')
BEGIN 
	DROP TABLE f.OrderPerf;
END

GO
CREATE TABLE f.OrderPerf(
	OrderID int NOT NULL,
	fkCalendar int NOT NULL,
	fkCustomer int NOT NULL,
	fkShipper int NULL,
	fkEmployee int NULL,
	Freight money NOT NULL,
	DaysToShip int NULL,
	DaysTilRequired int NULL,
	OrderCount int NOT NULL
);

-- Could use PRIMARY or UNIQUE for the ORDERID - just used to not load dupes

ALTER TABLE f.OrderPerf
ADD CONSTRAINT PK_ORD PRIMARY KEY(OrderID)
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoCAL
	FOREIGN KEY (fkCalendar)              -- FROM the LOCAL TABLE
	 REFERENCES  dim.Calendar(pkCalendar) -- TO the FOREIGN TABLE
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoCUST
	FOREIGN KEY (fkCustomer)              -- FROM the LOCAL TABLE
	 REFERENCES  dim.Customers(pkCustomer) -- TO the FOREIGN TABLE
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoSHIP
	FOREIGN KEY (fkShipper)               --FROM the LOCAL TABLE
	 REFERENCES dim.Shipper(pkShipId)     --TO the FOREIGN TABLE
;

ALTER TABLE f.OrderPerf
ADD CONSTRAINT FK_ORDtoEMP
	FOREIGN KEY (fkEmployee)              --FROM the LOCAL TABLE         
	 REFERENCES dim.Employees(EmployeeID) --TO the FOREIGN TABLE
;


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.094

### **_Simplicity :_** 

### **_The f.ProductPerf table is a fact table capturing product performance metrics, with a primary key on OrderID and ProductID._ _Foreign key constraints link it to dimension tables (dim.Calendar, dim.Products, dim.Customers) for time, product, and customer details._**

### **_fact table 2nd -_**  **_f.Productperf_**

In [42]:
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'ProductPerf')
BEGIN 
	DROP TABLE f.ProductPerf;
END

CREATE TABLE f.ProductPerf(
	OrderID int NOT NULL,
	OrderDate int NOT NULL,
	ProductID int NOT NULL,
	CustomerID int NOT NULL,
	UnitPrice money NOT NULL,
	Quantity smallint NOT NULL,
	Discount float NOT NULL,
	DiscFlag int NOT NULL,
	LineTotal money NULL
);

-- Adds primary constraints & Foreign constraints to the table.

ALTER TABLE f.ProductPerf                                           -- FROM the LOCAL TABLE
ADD CONSTRAINT PK_ProdPerf PRIMARY KEY(OrderID, ProductID)          --TO the FOREIGN TABLE

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoCAL
	FOREIGN KEY (OrderDate)                                     -- FROM the LOCAL TABLE
	 REFERENCES dim.Calendar(pkCalendar)                        --TO the FOREIGN TABLE
;

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoPROD
	FOREIGN KEY (ProductID)                                      -- FROM the LOCAL TABLE
	 REFERENCES dim.Products(pkProdId)                           --TO the FOREIGN TABLE
;

ALTER TABLE f.ProductPerf
ADD CONSTRAINT FK_PRODtoCUST
	FOREIGN KEY (CustomerID)                                      -- FROM the LOCAL TABLE
	 REFERENCES dim.Customers(pkCustomer)                         --TO the FOREIGN TABLE
;


Commands completed successfully.

Total execution time: 00:00:00.031

### **_Simplicity :_**  

### <span style="font-size: 16.38px;"><b><i>The f.ProductPerf table is a hub for tracking product performance, uniquely identified by order and product IDs. It connects seamlessly to time, product, and customer details stored in related tables (dim.Calendar, dim.Products, dim.Customers).&nbsp;</i></b></span>